In [3]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import pandas as pd
import mysql.connector
from dotenv import load_dotenv

# Load credentials from .env
load_dotenv()

db_host = os.getenv("DB_HOST")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_name = os.getenv("DB_NAME")

# Read the CSV file
df = pd.read_csv("cleaned_data/happiness_cleaned.csv")

# Connect to MySQL
conn = mysql.connector.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name
)
cursor = conn.cursor()

# Insert countries (ignore duplicates)
country_insert = """
    INSERT IGNORE INTO countries (country_name, region)
    VALUES (%s, %s)
"""
countries = df[["country", "region"]].drop_duplicates().values.tolist()
cursor.executemany(country_insert, countries)
conn.commit()

# Fetch country_id mapping
cursor.execute("SELECT country_id, country_name FROM countries")
country_map = {name: cid for cid, name in cursor.fetchall()}

# Prepare happiness data with mapped country_id
records = []
for _, row in df.iterrows():
    country_id = country_map.get(row["country"])
    if country_id:
        records.append((
            country_id,
            int(row["year"]),
            float(row["happiness_score"]),
            float(row["gdp"]),
            float(row["social_support"]),
            float(row["life_expectancy"]),
            float(row["freedom"]),
            float(row["corruption"]),
            float(row["generosity"])
        ))

# Insert happiness data
data_insert = """
    INSERT IGNORE INTO happiness_data (
        country_id, year, happiness_score,
        gdp, social_support, life_expectancy,
        freedom, corruption, generosity
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
cursor.executemany(data_insert, records)
conn.commit()

cursor.close()
conn.close()

print("Data load complete.")


Data load complete.
